In [ ]:
from check_accuracy import check_result
from create_mini_batch import random_mini_batches
from initialize_parameters import initialize_adam, initialize_parameters
from forward_propagation import forward_propagation
from cost_function import cost_function
from back_propagation import back_propagation
from activate_and_derivative import derivative_tanh, sigmoid, tanh
from layer_size import layer_size
from update_learning_rate import learning_rate_update
from update_parameters import update_parameters
import numpy as np


def load_parameters(link):
    parameters = np.ndarray.item(np.load(link, allow_pickle="True"))
    return parameters


def brain(X, Y, X_dev, Y_dev, learning_rate, lambd, hidden_layer, initialize_para = True):
    """
    Arguments:
    X -- trainset of shape (size_of_picture*3, number of examples)
    Y -- labels of shape (1, number of examples)
    X_dev -- devset of shape (size_of_picture*3, number of examples)
    Y_dev -- devlabels of shape (1, number of examples)
    parameters -- python dictionary of initialize parameters, or previous parameters 
    lambd -- regularization parameters

    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    # initialize
    layers, L = layer_size(X, Y, hidden_layer)
    if initialize_para == True:
        parameters = initialize_parameters(layers, L)
        accuracy = 0
    else:
        parameters = load_parameters("parameters_dev.npy")
        A, _ = forward_propagation(X_dev, L, load_parameters("parameters_dev.npy"))
        accuracy = check_result(A, Y_dev)

    parameters_adam = initialize_adam(parameters, L)
    decay_rate = 0.2
    iteration = 0
    accuracy = 0
    cost = 100000

    while True:

        mini_batches_X, mini_batches_Y = random_mini_batches(X, Y)
        num_mini_batch = len(mini_batches_X)

        for i in range(num_mini_batch):
            _, cache = forward_propagation(mini_batches_X[i], L, parameters)

            # learning_rate = learning_rate_update(learning_rate, decay_rate, iteration)
            grads = back_propagation(parameters, cache, mini_batches_X[i], mini_batches_Y[i], L)
            parameters, parameters_adam = update_parameters(
                parameters, grads, learning_rate, L, iteration, parameters_adam)


        if iteration % 50 == 0 :
            A_L, _= forward_propagation(X, L, parameters)
            print("cost at iterations ", iteration, " is ",
                  cost_function(A_L, Y, lambd, parameters, L))
            predict, _ = forward_propagation(X_dev, L, parameters)
            if check_result(predict, Y_dev) >= accuracy:
                accuracy = check_result(predict, Y_dev)
                print("accuracy of prediction: ", accuracy, ' %')
                print("save parameters to test")
                np.save('parameters_test.npy', parameters)
            if cost_function(A_L, Y, lambd, parameters, L) < cost:
                cost = cost_function(A_L, Y, lambd, parameters, L)
                print("save parameters to dev")
                np.save('parameters_dev.npy', parameters)

        iteration += 1
        if iteration > 500:
            break
    
